# Nhận diện khuôn mặt thời gian thực với FaceNet & MTCNN trên Webcam

Bài tập thực hành: Xây dựng hệ thống nhận diện khuôn mặt theo thời gian thực sử dụng:
- **MTCNN**: Phát hiện khuôn mặt
- **FaceNet**: Trích xuất đặc trưng và so sánh khuôn mặt
- **OpenCV**: Truy cập webcam

**Điều kiện so sánh:**
- Similarity > 0.7: "Matched"
- Similarity < 0.7: "Unknown"

In [90]:
import os
import cv2
import numpy as np
from mtcnn.mtcnn import MTCNN
from keras_facenet import FaceNet

# =========================
# CONFIG
# =========================
KNOWN_DIR = "known"          # thư mục chứa ảnh người đã biết
THRESHOLD = 0.7             # similarity threshold
CAM_INDEX = 0               # webcam index (0 thường là webcam mặc định)
MIN_FACE_SIZE = 60          # bỏ qua mặt quá nhỏ (px)

print("✅ Đã import các thư viện và thiết lập config")

✅ Đã import các thư viện và thiết lập config


In [91]:
# =========================
# HELPERS
# =========================
def l2_normalize(x: np.ndarray, eps: float = 1e-10) -> np.ndarray:
    """Chuẩn hóa vector theo L2"""
    return x / (np.linalg.norm(x) + eps)

def cosine_similarity(a: np.ndarray, b: np.ndarray) -> float:
    """Tính độ tương đồng cosine giữa 2 vectors"""
    a = l2_normalize(a)
    b = l2_normalize(b)
    return float(np.dot(a, b))

def preprocess_face(face_bgr: np.ndarray, target_size=(160, 160)) -> np.ndarray:
    """
    FaceNet thường dùng input 160x160, RGB.
    Trả về mảng (160,160,3) RGB uint8.
    """
    face_rgb = cv2.cvtColor(face_bgr, cv2.COLOR_BGR2RGB)
    face_rgb = cv2.resize(face_rgb, target_size, interpolation=cv2.INTER_AREA)
    return face_rgb

def get_embedding(embedder: FaceNet, face_rgb_160: np.ndarray) -> np.ndarray:
    """
    keras-facenet: embeddings() nhận list/np array ảnh RGB (uint8 ok).
    """
    emb = embedder.embeddings([face_rgb_160])[0]  # shape (512,)
    return emb.astype(np.float32)

def safe_crop(frame: np.ndarray, x: int, y: int, w: int, h: int) -> tuple:
    """Crop an toàn, đảm bảo không vượt biên"""
    H, W = frame.shape[:2]
    x1 = max(0, x)
    y1 = max(0, y)
    x2 = min(W, x + w)
    y2 = min(H, y + h)
    return frame[y1:y2, x1:x2], (x1, y1, x2, y2)

print("✅ Đã định nghĩa các helper functions")

✅ Đã định nghĩa các helper functions


In [92]:
# =========================
# LOAD MODELS
# =========================
print("Đang load models...")
detector = MTCNN()
embedder = FaceNet()  # load FaceNet
print("✅ MTCNN và FaceNet đã sẵn sàng!")

Đang load models...


Exception ignored in: <_io.BufferedReader>
Traceback (most recent call last):
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\lz4\frame\__init__.py", line 753, in flush
    self._fp.flush()
ValueError: I/O operation on closed file.
Exception ignored in: <_io.BufferedReader>
Traceback (most recent call last):
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\lz4\frame\__init__.py", line 753, in flush
    self._fp.flush()
ValueError: I/O operation on closed file.
Exception ignored in: <_io.BufferedReader>
Traceback (most recent call last):
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\lz4\frame\__init__.py", line 753, in flush
    self._fp.flush()
ValueError: I/O operation on closed file.


✅ MTCNN và FaceNet đã sẵn sàng!


In [93]:
# =========================
# BUILD KNOWN DATABASE
# =========================
known_embeddings = []
known_names = []

# Tạo thư mục nếu chưa có
if not os.path.isdir(KNOWN_DIR):
    os.makedirs(KNOWN_DIR, exist_ok=True)
    print(f"[!] Đã tạo thư mục '{KNOWN_DIR}'. Hãy bỏ ảnh người quen vào đó rồi chạy lại.")
else:
    image_files = [f for f in os.listdir(KNOWN_DIR) if f.lower().endswith((".jpg", ".jpeg", ".png"))]
    
    if len(image_files) == 0:
        print(f"[!] Thư mục '{KNOWN_DIR}' chưa có ảnh. Hãy thêm ảnh (jpg/png) rồi chạy lại.")
    else:
        print(f"[*] Đang tạo database embeddings từ thư mục {KNOWN_DIR}/ ...")
        
        for fn in image_files:
            path = os.path.join(KNOWN_DIR, fn)
            img = cv2.imread(path)
            if img is None:
                print(f"[!] Không đọc được: {path}")
                continue

            # detect face in known image
            faces = detector.detect_faces(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            if len(faces) == 0:
                print(f"[!] Không phát hiện mặt trong: {fn}")
                continue

            # lấy face có confidence cao nhất
            faces = sorted(faces, key=lambda d: d.get("confidence", 0), reverse=True)
            x, y, w, h = faces[0]["box"]
            face_crop, _ = safe_crop(img, x, y, w, h)

            if face_crop.size == 0 or min(face_crop.shape[:2]) < MIN_FACE_SIZE:
                print(f"[!] Mặt quá nhỏ/lỗi crop trong: {fn}")
                continue

            face_160 = preprocess_face(face_crop)
            emb = get_embedding(embedder, face_160)

            name = os.path.splitext(fn)[0]  # tên = filename không đuôi
            known_embeddings.append(emb)
            known_names.append(name)
            print(f"    + Loaded: {name}")

        known_embeddings = np.array(known_embeddings, dtype=np.float32)
        
        if len(known_embeddings) == 0:
            print("[!] Không tạo được embeddings nào từ known/. Hãy dùng ảnh rõ mặt hơn.")
        else:
            print(f"\n✅ Database sẵn sàng: {len(known_embeddings)} người")
            print(f"📋 Danh sách: {', '.join(known_names)}")

[*] Đang tạo database embeddings từ thư mục known/ ...
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
    + Loaded: Cristiano_Ronaldo_2275_(cropped)
[!] Không đọc được: known\ronaldo.png

✅ Database sẵn sàng: 1 người
📋 Danh sách: Cristiano_Ronaldo_2275_(cropped)


In [94]:
# =========================
# REALTIME WEBCAM
# =========================
if len(known_embeddings) == 0:
    print("⚠️  Không có database để so sánh. Hãy thêm ảnh vào thư mục 'known/' trước!")
else:
    cap = cv2.VideoCapture(CAM_INDEX)
    if not cap.isOpened():
        print("[!] Không mở được webcam.")
    else:
        print("[*] Webcam đã sẵn sàng. Nhấn 'q' để thoát.")
        print(f"[*] Threshold: {THRESHOLD}")
        print(f"[*] Số người trong database: {len(known_names)}")
        print("-" * 50)
        
        frame_count = 0
        
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            # Flip frame để tạo hiệu ứng gương
            frame = cv2.flip(frame, 1)
            
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            detections = detector.detect_faces(rgb)

            for det in detections:
                conf = det.get("confidence", 0)
                if conf < 0.90:  # bỏ qua detection có confidence thấp
                    continue

                x, y, w, h = det["box"]
                face_crop, (x1, y1, x2, y2) = safe_crop(frame, x, y, w, h)

                if face_crop.size == 0:
                    continue
                if min(face_crop.shape[:2]) < MIN_FACE_SIZE:
                    continue

                face_160 = preprocess_face(face_crop)
                emb_live = get_embedding(embedder, face_160)

                # So khớp: lấy similarity lớn nhất
                sims = [cosine_similarity(emb_live, e) for e in known_embeddings]
                best_idx = int(np.argmax(sims))
                best_sim = float(sims[best_idx])
                best_name = known_names[best_idx]

                # Kiểm tra threshold
                if best_sim > THRESHOLD:
                    label = "Matched"
                    color = (0, 255, 0)  # XANH LÁ
                    display_text = f"{best_name} | sim={best_sim:.2f}"
                else:
                    label = "Unknown"
                    color = (0, 0, 255)  # ĐỎ
                    display_text = f"Unknown | sim={best_sim:.2f}"

                # Vẽ bounding box
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                
                # Vẽ background cho text
                (tw, th), _ = cv2.getTextSize(display_text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
                cv2.rectangle(frame, (x1, y1-30), (x1+tw, y1), color, -1)
                
                # Vẽ text
                cv2.putText(frame, display_text, (x1, y1-10), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                
                # Vẽ confidence
                cv2.putText(frame, f"Conf: {conf:.2f}", (x1, y2+20),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            # Hiển thị hướng dẫn
            cv2.putText(frame, "Press 'q' to quit", (10, 30),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            
            # Hiển thị frame
            cv2.imshow("FaceNet + MTCNN Realtime", frame)
            
            frame_count += 1
            
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break

        cap.release()
        cv2.destroyAllWindows()
        print("\n✅ Đã đóng webcam và giải phóng tài nguyên")
        print(f"📊 Tổng số frames xử lý: {frame_count}")

[*] Webcam đã sẵn sàng. Nhấn 'q' để thoát.
[*] Threshold: 0.7
[*] Số người trong database: 1
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step
1/1 ━━━━━━━━━━

## 📝 Giải thích Logic Chính Xác

### 🔍 Luồng xử lý:

1. **Load Models**
   - MTCNN: Detector phát hiện khuôn mặt
   - FaceNet: Trích xuất embedding 512 chiều

2. **Build Database** 
   - Đọc ảnh từ thư mục `known/`
   - Detect face bằng MTCNN
   - Trích xuất embedding bằng FaceNet
   - Lưu vào arrays `known_embeddings` và `known_names`

3. **Real-time Recognition**
   - Capture frame từ webcam
   - Detect faces trong frame (MTCNN)
   - Với mỗi face:
     - Crop và resize về 160x160
     - Trích xuất embedding
     - Tính cosine similarity với tất cả known embeddings
     - Lấy similarity cao nhất
     - **Điều kiện**: 
       - `similarity > 0.7` → **Matched** (XANH LÁ) + hiển thị tên
       - `similarity ≤ 0.7` → **Unknown** (ĐỎ)

### ⚙️ Các tham số quan trọng:

- **THRESHOLD = 0.7**: Ngưỡng phân biệt matched/unknown
- **MIN_FACE_SIZE = 60px**: Bỏ qua face quá nhỏ
- **Confidence MTCNN > 0.90**: Chỉ xử lý detection có confidence cao